# Open AI Search 기능 추가

## 라이브러리 import

In [2]:
import requests

In [ ]:
# api-key 변수에 저장
api_key = ""

## REST API 정보 입력
curl, json 형식으로 가져옴

In [ ]:
# endpoint
endpoint = "]"
# method : POST

# headers
headers = {
    "Content-Type" : "application/json",
    "api-key" : api_key
}

# 임시 프롬프트
prompt = "남해 관광지 3곳만 알려줘"

# body
body = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 남해 여행 전문가야"
        }
      ]
    }, 
    {
      "role": "user",
      "content": prompt
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

In [ ]:
# API 엔드포인트에 POST 요청을 보내고 응답을 받음
response = requests.post(endpoint, headers=headers, json=body)

# 응답을 JSON 형식으로 변환
response_json = response.json()

# 응답에서 메시지 및 콘텐츠 필터링 결과 추출
message = response_json['choices'][0]['message']
content_filter = response_json['choices'][0]['content_filter_results']

# 메시지에서 역할(role)과 내용(content) 가져오기
role = message['role']
content = message['content']

# 응답 상태 코드와 응답 본문을 출력하는 디버깅 코드 (주석 해제하면 확인 가능)
# print(response.status_code, response.text)


print(role)

assistant


In [3]:
from IPython.display import Markdown

display(Markdown(content))

NameError: name 'content' is not defined

## 함수로 만들기

In [1]:
import requests
import re

In [ ]:
def request_gpt (prompt):
    # headers
    headers = {
        "Content-Type" : "application/json",
        "api-key" : api_key
    }

    # body
    body = {
        "messages": [
            {
            "role": "system",
            "content": [
                {
                "type": "text",
                "text": "너는 남해 여행 전문가야"
                }
            ]
            }, 
            {
            "role": "user",
            "content": prompt
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800,
        
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": ai_search_endpoint,
                    "index_name": ai_search_index,
                    "semantic_configuration": ai_search_semantic,
                    "query_type": "semantic",
                    "fields_mapping": {},
                    "in_scope": True,
                    "role_information": "너는 남해 여행 전문가야",
                    "filter": None,
                    "strictness": 3,
                    "top_n_documents": 5,
                    "authentication": {
                    "type": "api_key",
                    "key": ai_search_api_key
                    },
                    "key": ai_search_api_key,
                    "indexName": ai_search_index
                }
            }
        ],
    }
    # API 엔드포인트에 POST 요청을 보내고 응답을 받음
    response = requests.post(endpoint, headers=headers, json=body)
    
    print(response)
    
    if response.status_code == 200:

        # 응답을 JSON 형식으로 변환
        response_json = response.json()

        # 응답에서 메시지 및 콘텐츠 필터링 결과 추출
        message = response_json['choices'][0]['message']
        #content_filter = response_json['choices'][0]['content_filter_results']
        citaiton_list = message['context']['citations']

        # 메시지에서 역할(role)과 내용(content) 가져오기
        role = message['role']
        content = message['content']
        content = re.sub(r'\[doc(\d+)\]', r'[참조 \1]', content)

        print(role)
        return content, citaiton_list
    
    else:
        return ""

# 함수 불러오기
content, citation_list = request_gpt("남해 관광지 3곳만 알려줘")

    # 응답 상태 코드와 응답 본문을 출력하는 디버깅 코드 (주석 해제하면 확인 가능)
    # print(response.status_code, response.text)
from IPython.display import Markdown
display(Markdown(content))

for index in range(len(citation_list)):
    c = citation_list[index]
    print("Index : {}".format(index + 1), c['content'])
    